# LCA with foreground

This notebook demonstrates how to include foregrounds system into LCA using this library.

### Preparation

1. Navigate to the project that contains the ecoinvent database (ecoinvent database is not necessary if you don't need to find characterization factors through Brightway).

In [1]:
import bw2data as bd

bd.projects.set_current("advlca25")

bd.databases

Databases dictionary with 4 object(s):
	ALIGNED-biob-prod-dummy
	ecoinvent-3.11-biosphere
	ecoinvent-3.11-consequential
	exldb

2. Import required libraries

In [2]:
import os
from bamboo_lca.background_importer import *
from bamboo_lca.foreground_importer import *
from bamboo_lca.datapackage_builder import *
from bamboo_lca.uncertainty_handler import *
from bamboo_lca.lca_wrapper import *
from bamboo_lca.uncertainty_importer import *
from bamboo_lca.utils import *

3. Define required constants

In [3]:
# BACKGROUND DATABASE FILE PATH
EXIOBASE_AGGREGATED_A_FILE = os.path.join(os.getcwd(), "data/A.txt")
EXIOBASE_AGGREGATED_S_FILE = os.path.join(os.getcwd(), "data/S.txt")

# FOREGROUND DATABASE FILE PATH
FOREGROUND_FILE = os.path.join(os.getcwd(), "data/foreground_system.csv")

# CHARACTERIZATION FACTOR MAPPING FILE PATH
CF_MAPPING_FILE = os.path.join(os.getcwd(), "data/cf_mapping_file.csv")

# LCIA METHOD
METHOD = ('ecoinvent-3.11', 'IPCC 2013', 'climate change', 'global temperature change potential (GTP100)')

# CHOSEN BIOSPHERE EMISSIONS
GHG = ["CO2 - combustion - air",
        "CO2 - non combustion - Cement production - air",
        "CO2 - non combustion - Lime production - air",
        "CO2 - waste - fossil - air",
        "CH4 - agriculture - air",
        "CH4 - waste - air",
        "CH4 - combustion - air",
        "CH4 - non combustion - Extraction/production of (natural) gas - air",
        "CH4 - non combustion - Extraction/production of crude oil - air",
        "CH4 - non combustion - Mining of antracite - air",
        "CH4 - non combustion - Mining of bituminous coal - air",
        "CH4 - non combustion - Mining of coking coal - air",
        "CH4 - non combustion - Mining of lignite (brown coal) - air",
        "CH4 - non combustion - Mining of sub-bituminous coal - air",
        "CH4 - non combustion - Oil refinery - air",
        "N2O - combustion - air",
        "N2O - agriculture - air",
        "SF6 - air"]

### Step 1: Get the background database matrices

1. Get technosphere matrix

In [4]:
bg_importer = BackgroundImporter()

tech_df = pd.read_table(EXIOBASE_AGGREGATED_A_FILE, sep='\t', header=None, low_memory=False)
raw_tech = tech_df.iloc[3:, 2:].astype('float').to_numpy()
tech_matrix = bg_importer.build_tech_matrix(raw_tech)

print(f"The shape of technosphere matrix is: {tech_matrix.shape}")

The shape of technosphere matrix is: (76, 76)


2. Get biosphere matrix

In [5]:
bio_df = pd.read_csv(EXIOBASE_AGGREGATED_S_FILE, header=[0,1], index_col=[0], sep='\t', low_memory=False)
bio_matrix = bg_importer.build_bio_matrix(bio_df, GHG)

print(f"The shape of biosphere matrix is: {bio_matrix.shape}")

The shape of biosphere matrix is: (18, 76)


3. Get characterization factor matrix  
The characterization factors can be extracted from the ecoinvent biosphere database or from `CF_MAPPING_FILE` file.

- If you already have required characterization factors, then you just need to add the characterization factors to the `CFs` column of `CF_MAPPING_FILE` file. Then, run Option 1.
- Otherwise, you need to extract characterization factors from the ecoinvent biosphere database. In this case, you must have the ecoinvent databases imported into Brightway. Then, run Option 2.

Option1: Get characterization factor matrix from the file directly.

In [6]:
# get characterization factor matrix from the file directly.
cf_matrix = bg_importer.build_cf_matrix(CF_MAPPING_FILE, GHG)  # By default, source="cf", so you don't need to pass this parameter to the function.

# print the diagonal to check the values.
print(f"The diagonal values of characterization factor matrix: \n {cf_matrix.diagonal()}")

All characterization factors have been found.
The diagonal values of characterization factor matrix: 
 [1.00e+00 1.00e+00 1.00e+00 1.00e+00 2.70e+01 2.98e+01 2.98e+01 2.98e+01
 2.98e+01 2.98e+01 2.98e+01 2.98e+01 2.98e+01 2.98e+01 2.98e+01 2.73e+02
 2.73e+02 2.52e+04]


Option2: Get characterization factor matrix from Brightway.

In [7]:
# get characterization factor matrix from code.
cf_matrix = bg_importer.build_cf_matrix(CF_MAPPING_FILE, GHG, "ecoinvent-3.11-biosphere", METHOD, source="code")

# print the diagonal to check the values.
print(f"The diagonal values of characterization factor matrix: \n {cf_matrix.diagonal()}")

All characterization factors have been found.
The diagonal values of characterization factor matrix: 
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
 4.30000000e+00 5.70000000e+00 5.70000000e+00 5.70000000e+00
 5.70000000e+00 5.70000000e+00 5.70000000e+00 5.70000000e+00
 5.70000000e+00 5.70000000e+00 5.70000000e+00 2.34200000e+02
 2.34200000e+02 2.82148093e+04]


### Step 2: Import the foreground database

4. Check out the user input file.

In [8]:
fg_tech_df = pd.read_table(FOREGROUND_FILE, sep=',')
fg_tech_df.head()

,Activity name,Exchange name,Exchange type,Exchange amount
0,column_1,EU28-Agriculture-Forestry-Fishing,technosphere,0.000000
1,column_1,EU28-Energy,technosphere,NaN
2,column_1,EU28-Natural gas and services related to natur...,technosphere,NaN
3,column_1,EU28-Industry,technosphere,0.000001
4,column_1,EU28-Motor Gasoline,technosphere,-0.000016


5. Get activities in background system.

In [9]:
bg_activities = get_bg_activities(EXIOBASE_AGGREGATED_A_FILE, "\t")
bg_activities[:5] # only list the first five activities here

['EU28-Agriculture-Forestry-Fishing',
 'EU28-Energy',
 'EU28-Natural gas and services related to natural gas extraction, excluding surveying',
 'EU28-Industry',
 'EU28-Motor Gasoline']

6. Get activities in foreground system.

In [10]:
fg_activities = get_fg_activities(FOREGROUND_FILE, ",", bg_activities)
fg_activities

['column_2', 'column_1', 'column_3']

7. Get foreground system matrices.  
The foreground system is constructed from four matrices: `fgbg`, `fgfg`, `bgfg`, and `bifg`. These matrices are named to reflect their row and column positions. Specifically:

- `fgbg`: This is the matrix locate the foreground row first, then the background column. It indicates the amount of exchange that takes from background to foreground. Normally, there is no such exchange. So, by default this matrix is all zero.
- `fgfg`: This is the matrix locate the foreground row first, then the foreground column. It indicates the amount of exchange that takes from foreground to foreground.
- `bgfg`: This is the matrix locate the background row first, then the foreground column. It indicates the amount of exchange that takes from foreground to background.
- `bifg`: This is the matrix locate the biosphere emission row first, then the foreground column. It indicates the amount of exchange that takes from foreground to biosphere.

In [11]:
fg_dataframe = get_fg_dataframe(fg_tech_df, fg_activities)

fg_importer = ForegroundImporter()

fgbg, fgfg, bgfg, bifg = fg_importer.extend_matrix(fg_dataframe, GHG, fg_activities, bg_activities)

8. Concatenate foreground system with background system

In [12]:
full_tech_matrix, full_bio_matrix = fg_importer.concatenate_matrix(tech_matrix, bio_matrix, fgbg, fgfg, bgfg, bifg)

# Print to compare the matrix before and after concatenate with foreground system
print(f"Technosphere shape change from {tech_matrix.shape} to {full_tech_matrix.shape}.")
print(f"Biosphere shape change from {bio_matrix.shape} to {full_bio_matrix.shape}.")

Technosphere shape change from (76, 76) to (79, 79).
Biosphere shape change from (18, 76) to (18, 79).


### Step 3: Build datapackage

9. Build datapackage

In [13]:
dp_builder = DatapackageBuilder()
datapackage_data = dp_builder.prepare_dp_matrix(full_tech_matrix, full_bio_matrix, cf_matrix)
datapackage = dp_builder.prepare_datapackage(datapackage_data)

### Step 4: Run the simulation

10. Choose functional unit

In [14]:
# This is the list includes all activities.
activities = fg_activities + bg_activities 

# We want to set "column_1" as functional unit.
selected_activity = "column_1"

# We need to find the index of the activity in activities.
index = activities.index(selected_activity)

# Define the index of the functional unit
functional_unit = {index: 1}
print(f"The functional unit is {functional_unit}")

The functional unit is {1: 1}


11. Run static simulation

In [15]:
lca = bc.LCA(
            demand=functional_unit,
            data_objs=[datapackage],
        )
lca.lci()
lca.lcia()

print(f"Brightway calculated lca score: {activities[index]}, {lca.score}")

Brightway calculated lca score: column_1, -78.83331086412699


### Step 5: Compare to manually LCA

12. Compare with lca manually.

In [16]:
lca_wrapper = LCAWrapper()

full_tech_matrix_manual = full_tech_matrix.copy()

np.fill_diagonal(full_tech_matrix_manual, -full_tech_matrix_manual.diagonal())
full_tech_matrix_manual = -full_tech_matrix_manual

manual_lca = lca_wrapper.manual_lca(full_tech_matrix_manual, full_bio_matrix, cf_matrix, index)
print(f"Manually calculated lca score: {activities[index]}, {manual_lca}")

Manually calculated lca score: column_1, -78.83331086412697
